# How to use this notebook
Throughout this notebook I will use pickle to save and load objects which take a long time to generate. [Read more about pickle](https://docs.python.org/3/library/pickle.html). This will save significant time by avoiding having to recreate these objects every time. If you choose to, you can [download the project files](https://adumb-files.s3.us-west-2.amazonaws.com/project_files.zip) I created in order to load the objects from these files. If you prefer, you can also generate these objects from scratch. At each stage, you will have the option to load the object from the file or create the object and then save it to a file. You will see where there are some places where you may prefer to use the pre generated objects, and places where you may want to create these objects from scratch in order to customize them.

The notebook goes in order of how to create the graph from scratch. However, if you are using the project files and just want to generate the visual representation of the graph, you simply need to run these four cells: [Import](#import), [Load layout](#layout), [Load pre-styled graph](#styled_graph), and [Visualize graph](#visualize).

# Imports <a id='import'></a>

In [ ]:
import csv
import igraph as ig
import leidenalg as la
import pickle
import colorsys
import random
import math

# Remove self links and duplicate links from links file
***NOTE: If you are using the project files then you can skip this step. If you scraped the Wikipedia dumps to generate the links_raw.csv file then you need to do this step.***

Sometimes Wikipedia pages will link to themselves, or they will contain multiple links to the same article. This is not the behavior we want when graphing the articles, so we will create a refined links csv which does not have any duplicate or self links. This will also allow us to identify disguised dead ends, articles which appeared to have a link, but end up being dead ends once their self links are removed. We will append the disguised dead ends to the dead ends file.

In [ ]:
disguised_deadends = []

with open('links_raw.csv') as links_file:
    with open ('links.csv', 'w') as new_links_file:
        reader = csv.reader(links_file)
        next(reader, None)
        writer = csv.writer(new_links_file)
        writer.writerow(['source', 'target'])
        
        cur_page = ''
        cur_links = set()
        # Iterate over every link in the csv
        for row in reader:
            if cur_page != row[0]:
                if len(cur_links) == 0:
                    disguised_deadends.append(cur_page)
                cur_links.clear()
                cur_page = row[0]
        
            # Skip if duplicate link
            if row[1] in cur_links:
                continue
            # Skip if self link
            if row[1] == row[0]:
                continue
            
            # Write link to new csv
            cur_links.add(row[1])
            writer.writerow([row[0], row[1]])

# Remove first element of disguised_deadends since it will be an empty string
disguised_deadends.pop(0)
print(disguised_deadends)
                    
with open('deadends.txt', 'a') as deadends_file:
    for page in disguised_deadends:
        deadends_file.write(f'\n{page}')

# Associate each page with a unique ID
We will create a bidirectional dictionary which will associate every page with a unique ID and allow us to look up the ID given the page name and vice versa.
### Load page_ids dictionary from file using pickle
If you are using the project files, you can run this line, otherwise you will need to create and save the page_ids dictionary before you can run this.

In [ ]:
with open('page_ids.pkl', 'rb') as page_file:
    page_ids = pickle.load(page_file)

### Create the page_ids dictionary
You only need to do this if you haven't already loaded the page_ids dictionary from a saved file.

In [ ]:
page_count = 0
page_ids = {}
with open('links.csv') as links_file:
    reader = csv.reader(links_file)
    next(reader, None)

    for row in reader:
        # Add any page to the dictionary which has not already been found
        if row[0] not in page_ids:
            page_ids[row[0]] = page_count
            page_ids[page_count] = row[0]
            page_count += 1
        if row[1] not in page_ids:
            page_ids[row[1]] = page_count
            page_ids[page_count] = row[1]
            page_count += 1

        # Print progress
        if page_count % 10_000 == 0:
            print(f'\rPages processed: {page_count}', end='')

    print()
    print(page_count)

### Save page_ids dictionary to file
You only need to run this once after creating the dictionary. Afterwards you should load the dictionary from the saved file.

In [ ]:
with open('page_ids.pkl', 'wb') as page_file:
    pickle.dump(page_ids, page_file)

# Create a list of edges for the graph
Create an edge for every link in the csv file. These edges will be used to create the graph.

In [ ]:
edges = []

with open('links.csv') as links_file:
    reader = csv.reader(links_file)
    next(reader, None)

    link_count = 0
    for row in reader:                    
        edges.append((page_ids[row[0]], page_ids[row[1]]))
        
        # Print progress
        link_count += 1
        if link_count % 100_000 == 0:
            print(f'\rLinks processed: {link_count}', end='')

# Create the graph object
We can now create the graph. This is all you need if you want to start doing analysis of the graph, like finding the shortest path between two objects. However, if you want to create a visual representation of the graph, more work is still needed.
### Load graph using pickle

In [ ]:
with open('graph.pkl', 'rb') as graph_file:
    graph = pickle.load(graph_file)

### Create graph from edges

In [ ]:
node_count = int(len(page_ids)/2)
graph = ig.Graph(node_count, edges=edges, directed=True)

### Save the graph object

In [ ]:
with open('graph.pkl', 'wb') as graph_file:
    pickle.dump(graph, graph_file)

# Generate the layout for the graph
We will now need to generate the layout for the graph. The layout determines where each node will be placed on the graph. The layout algorithm I used is the Distributed Recursive Layout (DRL), which will attempt to cluster closely linked articles together. [Read more about the layout algorithms](https://python.igraph.org/en/stable/tutorial.html#layout-algorithms)

***Note: This is probably the most resource intensive part of the entire project. It took about 4-5 days for the layout to finish generating***

### Load layout using pickle <a id='layout'></a>

In [ ]:
with open('layout.pkl', 'rb') as layout_file:
    layout = pickle.load(layout_file)

### Generate layout from scratch

In [ ]:
layout = graph.layout('drl')

### Save the layout object

In [ ]:
with open('layout.pkl', 'wb') as layout_file:
    pickle.dump(layout, layout_file)

# Detect communitites within the graph
At this point we could create the visual representation of the graph, but first we should make it more interesting by coloring each of the communities within the graph a different color. To do this, we first need to detect the diffent communities within the graph. This will be done using the [Leiden algortithm](https://leidenalg.readthedocs.io/en/stable/intro.html).

### Load community list using pickle

In [ ]:
with open('partition_list.pkl', 'rb') as partition_file:
    partition = pickle.load(partition_file)

### Calculate communities from scratch

In [ ]:
partition = la.find_partition(graph, la.ModularityVertexPartition)

### Create community list
Turns the partition object into a python list

In [ ]:
partition_list = []
for p in partition:
    partition_list.append(p)

### Save the partition object

In [ ]:
with open('partition_list.pkl', 'wb') as partition_file:
    pickle.dump(partition_list, partition_file)

### Create community dictionary
This will allow us to look up which community each article is in

In [ ]:
partition_dict = {}
for i in range(len(partition)):
    for v in partition[i]:
        partition_dict[v] = i

# Style the nodes
Now that we've determined which community each article belongs to, we can color the nodes of the graph according to their corresponding community. We will also resize each node according to the number of incoming links to its corresponding article using `indegree()`. The formulas to calculate the size and border width of each node were determined through a lot of trial and error.

### Load pre-styled graph <a id='styled_graph'></a>
If you load the pre-styled graph then you can skip the steps for styling the nodes and edges of the graph and skip to [visualizing the graph](#visualize).

In [ ]:
with open('graph_styled.pkl', 'rb') as graph_file:
    graph = pickle.load(graph_file)

### Create different colors for each community
This cell creates 44 different colors for each of the 44 communities. If you generate the communities from scratch, it may result in more or less than 44 communities, in which case you should adjust this cell as necessary. The colors were determined through trial and error to try and avoid very similar colors from being next to each other. The colors were first defined in hsv (hue, saturation, value) and then converted to rgb. The opacity of the verticies are 80% (0.8) and the edges are 5% (0.05). The reason for the low edge opacity is because of the large number of edges, otherwise it would be too cluttered to look at.

In [ ]:
vertex_colors = []
edge_colors = []
hues = [262, 229, 196, 164, 131, 98, 65, 33, 0, 327, 295]
saturations = [300, 567, 700, 433]
for s in saturations:
    for h in hues:
        hue = h/359
        saturation = s/1000
        rgb_color = colorsys.hsv_to_rgb(hue, saturation, 1)
        vertex_colors.append(f'rgba({round(rgb_color[0]*255)}, {round(rgb_color[1]*255)}, {round(rgb_color[2]*255)}, 0.8)')
        edge_colors.append(f'rgba({round(rgb_color[0]*255)}, {round(rgb_color[1]*255)}, {round(rgb_color[2]*255)}, 0.05)')

### Set color and size of nodes

In [ ]:
for i in range(len(partition)):
    for vertex in partition[i]:
        graph.vs[vertex]['color'] = vertex_colors[i]
        graph.vs[vertex]['frame_color'] = 'rgba(0, 0, 0, 0.5)'
        
        size = 168 * math.log10(0.00005 * graph.vs[vertex].indegree() + 1) + 3
        graph.vs[vertex]['size'] = size
        frame_size = 0.1 + (6.56/197) * (size-3)
        graph.vs[vertex]['frame_width'] = frame_size

# Simplify the graph
Now that we have calculated the layout, communities, and size of the nodes, let's simplify the graph. The graph has nearly 200,000,000 links, which will look horribly cluttered when visualized. It will also take a very long time to create the visualization. To simplify the graph, I only graphed 10% of the links and also removed links which go from one community to a different community. You can experiment with exactly how many links you want to graph and if you want to keep links that go between communities. However, from my testing, the best way to visualize the structure of the graph without too much clutter is to reduce the number of edges that are drawn.

### Remove edges from graph

In [ ]:
graph.delete_edges()

### Select edges to add back to graph
This is where you could change the code to visualize different data. For example if you wanted to only visualize the edges in community #3, you could do:
```
if source_partition == 2 and target_partition == 2:
    edges.append((page_ids[row[0]], page_ids[row[1]]))
```
Or only graph edges that link to a specific article like the *United States*:
```
if row[1] == 'United States':
    edges.append((page_ids[row[0]], page_ids[row[1]]))
```

In [ ]:
edges = []

with open('links.csv') as links_file:
    reader = csv.reader(links_file)
    next(reader, None)

    link_count = 0
    for row in reader:
        # Print progress
        link_count += 1
        if link_count % 100_000 == 0:
            print(f'\rLinks processed: {link_count}', end='')
            
        source_partition = partition_dict[page_ids[row[0]]]
        target_partition = partition_dict[page_ids[row[1]]]

        # Only add edge if it starts and ends in the same community
        if source_partition == target_partition:
            # 10% chance to add link
            rand = random.randint(0, 99)
            if rand < 10:
                edges.append((page_ids[row[0]], page_ids[row[1]]))

### Add edges back to graph

In [ ]:
graph.add_edges(edges)

# Style the edges
Adds color to the edges according to the partition they are in.

If you wanted, you could also change the colors of the nodes here by doing something like:
```
source_node = graph.es[edge].source
target_node = graph.es[edge].target
graph.vs[source_node]['color'] = 'rgba(100, 100, 100, 0.8)'
graph.vs[target_node]['color'] = 'rgba(200, 200, 200, 0.8)'
```
This can be useful when you are graphing all the links to a specific article and you want the source nodes to be the same color.

In [ ]:
for edge in range(graph.ecount()):
    source_partition = partition_dict[graph.es[edge].source]
    target_partition = partition_dict[graph.es[edge].target]
    if source_partition == target_partition:
        graph.es[edge]['color'] = edge_colors[source_partition]

### Save the styled graph
The graph has now been fully styled and the particular styling of this graph can be saved.

In [ ]:
with open('graph_styled.pkl', 'wb') as graph_file:
    pickle.dump(graph, graph_file)

# Visualize the graph  <a id='visualize'></a>
Create a png image of the graph. This image will be very large (10800 x 19200 pixels) in order to be able to zoom in on the image and see each dot. This image will also be in a portrait orientation and should be rotated counterclockwise 90° after the image has been generated. The reasoning is that it looked better this way, but you can also choose to generate the image in landscape orientation by changing the bounding box to 19200 x 10800. [Read more about styling the graph](https://python.igraph.org/en/stable/tutorial.html#drawing-a-graph-using-a-layout).

***Note: The time to create the image varies on the number of edges you are trying to graph. If you try to graph all 200,000,000 edges it could take up to a day. Visualizing 10% of the edges could take about an hour***

In [ ]:
visual_style = {}

visual_style["bbox"] = (10800, 19200) # Dimension of the image
visual_style["margin"] = 50 # Margin of pixelsfrom the borders
visual_style["layout"] = layout
visual_style["background"] = 'rgba(0, 0, 0, 0)' # Transparent background
visual_style["edge_arrow_size"] = 0
visual_style["edge_width"] = 0.25
visual_style["vertex_order_by"] = 'size' # Draw nodes in order of their size, larger nodes will be drawn last

plot = ig.plot(graph, 'graph.png', **visual_style)

# Graph analysis
You should now have all the tools needed to visualize the graph and customize the graph to visualize particular data. The rest of the notebook will explain how to analyze the graph to extract certain data from the graph, similar to what was presented in the video

### Load graph with links
For the graph analysis you will need the graph will all of the links. Using the styled graph here will give inaccurate results

In [ ]:
with open('graph.pkl', 'rb') as graph_file:
    graph = pickle.load(graph_file)

# Community categories
Extract the top categories for a given community.

### Create category dictionary
This will create a dictionary that will allow us to look up all of the categories for a given page

In [ ]:
with open('categories.csv') as links_file:
    reader = csv.reader(links_file)
    next(reader, None)

    category_count = 0
    categories = {}
    for row in reader:
        if row[0] not in page_ids:
            continue
        page_id = page_ids[row[0]]
        if page_id in categories:
            categories[page_id].append(row[1])
        else:
            categories[page_id] = [row[1]]
        category_count += 1

        if category_count % 100_000 == 0:
            print(f'\rCategories processed: {category_count}', end='')

### Create partition category dictionary
This will create a dictionary for a specific community specified by `partition_idx`. This dictionary will count how many times each category appears in the given community.

In [ ]:
partition_categories = {}
partition_idx = 6
for page_id in partition[partition_idx]:
    if page_id not in categories:
        continue
    page_categories = categories[page_id]
    for category in page_categories:
        if category in partition_categories:
            partition_categories[category] += 1
        else:
            partition_categories[category] = 1

### Print the top 100 categories
Prints the top 100 categories for the given community as well as the number of times each category appears in the community. You can adjust the code to print more or less than the top 100 categories.

In [ ]:
sorted_categories = sorted(partition_categories.items(), key=lambda x: x[1], reverse=True)
print(sorted_categories[:100])

# Analyze incoming links

### Incoming links to a specific article

In [ ]:
print(graph.vs[page_ids['United States']].indegree())

### Create links_to_page dictionary
This dictionary will keep track of how many incoming links each page has. We will then sort the dictionary to get the most linked articles.

In [ ]:
links_to_page = {}
for vertex in range(graph.vcount()):
    links_to_page[page_ids[vertex]] = graph.vs[vertex].indegree()
    
sorted_links = sorted(links_to_page.items(), key=lambda x: x[1], reverse=True)

### Top 100 articles with most incoming links
Use the sorted link count to print the top 100 most linked to articles as well as how many times they are linked to. You can adjust the code to print more or less than the top 100 articles

In [ ]:
for page, count in sorted_links[:100]:
    print(f'{page.ljust(50, " ")} {count}')

# Dead ends & Orphans
We already have a list of dead ends in the `deadends.txt` file which was found when scraping the Wikipedia dumps.

### Find orphans
Note: this will not include dead end orphans since they are not on the graph. The list of dead end orphans should be appeneded to the list of orphans found in this cell to get a list of all dead end orphans

In [ ]:
orphans = []
for page in links_to_page:
    if links_to_page[page] == 0:
        orphans.append(page)

print(len(orphans))

### Find dead end orphans
These compares the list of dead ends to the articles in the graph. Since dead end orphans are not in the graph, any dead ends which are not in the graph are therefore dead end orphans.

In [ ]:
deadend_orphans = []
with open('deadends.txt') as deadends_file:
    for line in deadends_file:
        title = line.strip()
        if not title in links_to_page:
            deadend_orphans.append(title)
            
print(deadend_orphans)

# Paths between articles

### Find a shortest path between two articles
Finds a single path whose distance is the shortest between the two articles.

In [ ]:
start = page_ids['Hairy ball theorem']
end = page_ids['Pepsi fruit juice flood']

shortest_paths = graph.get_shortest_paths(start, to=end, output='vpath')

### Find all shortest paths between two articles
Alternatively, this will find every path between two articles whose distances are the shortest.

In [ ]:
shortest_paths = graph.get_all_shortest_paths(start, to=end, mode='out')

### Print shortest paths
This will print the shortest paths found in a human readable format.

In [ ]:
for path in shortest_paths:
    for i in range(len(path)-1):
        print(page_ids[path[i]], end=" -> ")
    print(page_ids[path[-1]])

### Graph shortest path
If you wanted to graph the shortest path, you should first remove all edges from the graph and color every node to be 100% transparent. Then you can run something like the following. Then you would create the image of the graph in the same way, but without setting the `edge_width` property on the `visual_style`.

In [ ]:
for vertex in shortest_paths[0]:
    graph.vs[vertex]['color'] = 'rgba(229, 145, 255, 1)'
    graph.vs[vertex]['size'] = 150
    graph.vs[vertex]['frame_width'] = 5
    graph.vs[vertex]['frame_color'] = 'rgba(0, 0, 0, 0.8)'

shortest_edges = []
for i in range(1, len(shortest_paths[0])):
    shortest_edges.append((shortest_paths[0][i-1], shortest_paths[0][i]))
    
graph.add_edges(shortest_edges)

for edge in range(graph.ecount()):
    graph.es[edge]['color'] = 'rgba(252, 255, 56, 1)'
    graph.es[edge]['width'] = 25

# Degrees of separation

### Create page_link_dict
This will create a dictionary which will allow us to look up the list of outgoing links from a given page.

In [ ]:
page_link_dict = {}

link_count = 0
with open('links.csv') as links_file:
    reader = csv.reader(links_file)
    next(reader, None)

    for row in reader:
        if row[1] not in page_link_dict:
            page_link_dict[row[1]] = []
            
        if row[0] in page_link_dict:
            page_link_dict[row[0]].append(row[1])
        else:
            page_link_dict[row[0]] = [row[1]]

        link_count += 1
        if link_count % 100_000 == 0:
            print(f'Links processed: {link_count}', end='\r')

### Find pages in each degree
Choose a starting page and then determine which pages are in each degree of separation starting from the 1st degree. You can change `degrees` to determine how many degrees of separation to go to.

In [ ]:
start_page = 'Pluto'
degrees_of_separation = {start_page: 0}
degrees = 10

for degree in range(1, degrees+1):
    print(f'Degree: {degree}')
    for page in page_link_dict:
        if page in degrees_of_separation and degrees_of_separation[page] == degree - 1:
            for link in page_link_dict[page]:
                if link not in degrees_of_separation:
                    degrees_of_separation[link] = degree

### Count articles in each degree of separation

In [ ]:
degree_counts = [0 for _ in range(degrees+1)]
for page in degrees_of_separation:
    degree_counts[degrees_of_separation[page]] += 1
print(degree_counts)

### Visualize degrees of separation
If you wanted to visualize the degrees of separation, you should first remove all edges from the graph and color every node to be 100% transparent. Then you could run something like the following. Then you would create the image of the graph in the same way.

In [ ]:
degree_colors = ['rgba(255, 77, 77, 0.8)', 'rgba(219, 255, 77, 0.8)', 'rgba(77, 255, 148, 0.8)', 'rgba(77, 148, 255, 0.8)', 'rgba(219, 77, 255, 0.8)', 'rgba(255, 184, 77, 0.8)', 'rgba(112, 255, 77, 0.8)', 'rgba(77, 255, 255, 0.8)', 'rgba(113, 77, 255, 0.8)', 'rgba(255, 77, 184, 0.8)']
for vertex in range(graph.vcount()):
    if page_ids[vertex] in degrees_of_separation:
        degree = degrees_of_separation[page_ids[vertex]]
        graph.vs[vertex]['color'] = degree_colors[degree-1]
        graph.vs[vertex]['frame_color'] = 'rgba(0, 0, 0, 0.5)'
        graph.vs[vertex]['size'] = 5
        graph.vs[vertex]['frame_width'] = 0.16
graph.vs[page_ids['Pluto']]['color'] = 'rgba(255, 77, 184, 0.8)'